# Advent of Code 2023 - Haskell

## Setup

First let's figure out how to read a file. In Haskell, io operations are separated from "pure" operations, so we have to "bind" the result with "<-" to get the actual contents:

In [1]:
file <- readFile "day1p1_test.txt"
file

"1abc2\npqr3stu8vwx\na1b2c3d4e5f\ntreb7uchet"

now that we have `file :: String`, we can split it into lines:

In [2]:
let input = lines file
input

["1abc2","pqr3stu8vwx","a1b2c3d4e5f","treb7uchet"]

# Day 1

## Part 1

In Day 1, Part 1, we are asked to find the *calibration value* hidden in each line:
> On each line, the calibration value can be found by combining the first digit and the last digit (in that order) to form a single two-digit number.

it's clear that the first thing we need to do is to filter out any non-digit characters. Luckily we have just the predicate in Prelude:

In [3]:
import Data.Char (isDigit)
isDigit '3'
isDigit 'a'

True

False

we can use this as an argument to `filter` like so:

In [4]:
filter isDigit "1ab2sd3"

"123"

lets map this to each line:

In [5]:
map (filter isDigit) input

["12","38","12345","7"]

Nice! now we need to get the first and last character of each line and make a new string out of it. I couldn't find a simpler way to do this: so let's write it in its own function:

In [6]:
cc :: String -> String
cc a = head a : [last a]

cc "1"
cc "123"

"11"

"13"

our function also works for single digits!

In [7]:
map (cc . filter isDigit) input

["12","38","15","77"]

to convert from a string to an integer, we can simply use `read`. note that this is not best practice, since we dont handle errors. Also, `read` requires a type to be specified, which is kind of confusing. to avoid that, let's put it in its own function `readInt`:

In [8]:
read "12" :: Integer

readInt a = read a :: Integer

12

let's map this to each line:

In [9]:
map (readInt . cc . filter isDigit) input

[12,38,15,77]

Finally, let's sum them:

In [10]:
sum $ map (readInt . cc . filter isDigit) input

142

our code seems to work! let's put it into a function so we can run it with the actual puzzle input:

In [37]:
file <- readFile "day1.txt"
let input = lines file

day1p1 x = sum $ map (readInt . cc . filter isDigit) x

day1p1 input

54450

Correct!

## Part 2

first let's read the test input:

In [57]:
file <- readFile "day1p2_test.txt"
let input = lines file
input

["two1nine","eightwothree","abcone2threexyz","xtwone3four","4nineeightseven2","zoneight234","7pqrstsixteen"]

this problem is a bit complicated because we are required to convert strings like "eightwothree" to "823". notice that because of the overlapping letters, a simple string substitution will not work. let's focus on "normalizing" this string by converting it to something like "eighttwothree", where there is no overlap, so we can do simple string substitution.

let's observe the main problem with naive substitution is that it removes information: "eightwothree" becomes "8wo3", and "two" is lost. a simple way to put back this information is to be *less destructive* when replacing strings. instead of replacing "eight" -> "8", lets do "eight" -> "e8t", where we **keep** the *first* and *last* letters, therefore no information is lost. the extra letters won't be a problem later, since we filter by `isDigit` anyway. thus our substitution table, or mapping, looks like:



In [58]:
-- we could generate this programatically...
let mapping = [("one","o1e"),("two","t2o"),("three","t3e"),("four","f4r"),("five","f5e"),("six","s6x"),("seven","s7n"),("eight","e8t"),("nine","n9e")]

To replace a substring with another in a string, I got this [very nice answer from stack overflow](https://stackoverflow.com/a/49261236):

In [59]:
import Data.List.Split (splitOn)
import Data.List (intercalate)

replace (from, to) = intercalate to . splitOn from

replace ("hello", "goodbye") "hello world"

"goodbye world"

take note that `splitOn` is not part of the standard library (*prelude*), so we need to install it with: `stack install split`. you can take a moment to see how this works, but it basically splits the string *on* the substring we want to replace, and puts the `to` string in those places.

Now that we have our `mapping` and our `replace`, all we need to do is apply this replace for each tuple of the mapping, so that all text is substituted with its digit. this would look like:

In [67]:
replaceAll x = replace ("one","o1e") . replace ("two","t2o") . replace ("three","t3e") -- (...)

but since we already have the mapping in a list form, we can use `foldr` to apply the function repeatedly:

In [68]:
replaceAll x = foldr replace x mapping

lets map this `replaceAll` for each line:

In [61]:
map replaceAll input

["t2o1n9e","e8t2ot3e","abco1e2t3exyz","xt2o1e3f4r","4n9ee8ts7n2","zo1e8t234","7pqrsts6xteen"]

looks correct! the rest is the same as *part 1*:

In [62]:
day1p2 x = sum $ map (readInt . cc . filter isDigit . replaceAll) x
day1p2 input


281

yay! now lets run it for the puzzle input:

In [63]:
file <- readFile "day1.txt"
let input = lines file

day1p2 input

54265

In [64]:
putStrLn "🎉 🎉 🎉"

🎉 🎉 🎉